In [2]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
num_classes=10
print(x_train.shape, y_train.shape)
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
input_shape = (28, 28, 1)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

(60000, 28, 28) (60000,)
x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
batch_size = 128
epochs = 10

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu',input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,optimizer=keras.optimizers.Adadelta(),metrics=['accuracy'])

D:\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
hist = model.fit(x_train, y_train,batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

model.save('mnist.h5')
print("Saving the model as mnist.h5")
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 49s 97ms/step - accuracy: 0.1275 - loss: 2.2919 - val_accuracy: 0.2057 - val_loss: 2.2558
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 44s 93ms/step - accuracy: 0.2218 - loss: 2.2536 - val_accuracy: 0.4283 - val_loss: 2.1814
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 45s 95ms/step - accuracy: 0.3538 - loss: 2.1765 - val_accuracy: 0.5863 - val_loss: 2.0224
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 47s 100ms/step - accuracy: 0.4934 - loss: 2.0016 - val_accuracy: 0.6892 - val_loss: 1.6784
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 47s 100ms/step - accuracy: 0.6218 - loss: 1.6451 - val_accuracy: 0.7705 - val_loss: 1.1743
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 97ms/step - accuracy: 0.6950 - loss: 1.2137 - val_accuracy: 0.8067 - val_loss: 0.8226
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 46s 98ms/step - accuracy: 0.7332 - loss: 0.9385 - val_accuracy: 0.8320 - val_loss: 0.6529
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 47s 100ms/step - accuracy: 0.7632 - loss: 0.7931

The model has successfully trained
Saving the model as mnist.h5
Test loss: 0.4695594608783722
Test accuracy: 0.8694000244140625


In [6]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict(img)[0]
    print(res)
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0
        
        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Draw..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command = self.classify_handwriting)   
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)
       
        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)
        
        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")
        
    def classify_handwriting(self):
        HWND = self.canvas.winfo_id()  # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND)  # get the coordinate of the canvas
        a,b,c,d = rect
        rect=(a+4,b+4,c-4,d-4)
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')
       
app = App()
mainloop()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
[3.6845782e-01 6.4480182e-10 9.1136850e-02 1.6472366e-02 5.7427009e-05
 4.7910142e-01 4.4955182e-04 3.7704824e-05 4.4256914e-02 2.9844581e-05]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
[4.6893466e-02 8.0280547e-09 3.5911787e-01 7.1613319e-02 1.2983398e-04
 4.3077114e-01 3.0913581e-03 4.1433330e-05 8.8262290e-02 7.9259051e-05]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[3.9467725e-01 1.1644489e-09 7.7454865e-02 2.9892243e-02 7.5976524e-05
 4.5570245e-01 2.6342759e-04 1.3059386e-04 4.1668594e-02 1.3460578e-04]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[1.8663575e-01 1.4438254e-08 7.3131430e-01 1.1105724e-02 7.1973977e-06
 5.1051795e-02 2.6321190e-04 1.0603281e-04 1.9504711e-02 1.1291435e-05]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
[3.4406632e-01 1.4022195e-09 1.6999519e-01 2.1438757e-02 1.7830725e-04
 4.0600431e-01 2.5326877e-03 5.2148338e-05 5.5678762e-02 5.3416010e-05]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
[3.1579193e-02 1.0577990e-08 5.9053093e-01 2.6618189e-01